CHGNet Formation Energy Calculation

In [4]:
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
from chgnet.model import CHGNet
from chgnet.model.dynamics import CHGNetCalculator

chgnet_model = CHGNet.load()

# Create the ASE calculator using the CHGNet model
calculator = CHGNetCalculator(chgnet_model)
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/Li_Srinibas.cif") 
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = calculator
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# # Let us do this for La, Zr, and O as well
# pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/La_Srinibas.cif")
# ase_atoms = adaptor.get_atoms(pmg_structure)
# ase_atoms.calc = calculator
# total_energy = ase_atoms.get_potential_energy()
# mu_model_La = total_energy / len(ase_atoms)
# print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/Zr_Srinibas.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = calculator
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/O2_Srinibas.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = calculator
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
CHGNet will run on cpu
Li: μ_model = -0.515714 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
Structure graph_id=None has 16 isolated atom(s) with atom_graph_cutoff=6. CHGNet calculation will likely go wrong


IndexError: too many indices for tensor of dimension 1

In [13]:
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from ase import Atoms
from ase.build import make_supercell
import numpy as np
from chgnet.model import CHGNet
from chgnet.model.dynamics import CHGNetCalculator

_adaptor = AseAtomsAdaptor()
_chgnet = CHGNet.load()

def _load_ase(path)->Atoms:
    s = Structure.from_file(path)
    s = Structure(lattice=s.lattice, species=s.species, coords=(s.frac_coords % 1.0), coords_are_cartesian=False)
    a = _adaptor.get_atoms(s); a.set_pbc([True,True,True]); a.wrap(); return a

def _min_d(a: Atoms):
    D = a.get_all_distances(mic=True)
    D[np.diag_indices_from(D)] = np.inf
    return float(np.nanmin(D))

# ---------- Option A: BULK-LIKE (recommended) ----------
def mu_bulk_like(cif_path, label="X", target_nn=3.0, nn_band=(2.5,3.2),
                 min_len=10.0, min_atoms=16, cutoff=8.0):
    a = _load_ase(cif_path)
    # Isotropic rescale to target NN
    for _ in range(12):
        dmin = _min_d(a)
        if np.isfinite(dmin) and nn_band[0] <= dmin <= nn_band[1]: break
        scale = 1.2 if (not np.isfinite(dmin) or dmin<=1e-6) else max(0.2, min(5.0, (target_nn/dmin)**0.7))
        a.set_cell(a.cell*scale, scale_atoms=True); a.wrap()
    # Enforce min cell length
    Lx,Ly,Lz = a.cell.lengths()
    s = max(min_len/Lx, min_len/Ly, min_len/Lz, 1.0)
    if s>1.0: a.set_cell(a.cell*s, scale_atoms=True); a.wrap()
    # Enforce min atoms via deterministic diagonal supercell
    n=len(a); nx=ny=nz=1
    while nx*ny*nz*n < min_atoms:
        if nx<=ny and nx<=nz: nx+=1
        elif ny<=nx and ny<=nz: ny+=1
        else: nz+=1
    a = make_supercell(a, [[nx,0,0],[0,ny,0],[0,0,nz]]); a.wrap()
    a.calc = CHGNetCalculator(_chgnet, atom_graph_cutoff=cutoff)
    E = a.get_potential_energy(); mu = E/len(a)
    print(f"{label}: μ_bulk-like = {mu:.6f} eV/atom  (dmin≈{_min_d(a):.2f} Å, cutoff={cutoff} Å, natoms={len(a)})")
    return mu

# ---------- Option B: ATOMIC (matches your −0.5157 style) ----------
def mu_atomic_like(cif_path, label="X", box=25.0, cutoff=1.5):
    a = _load_ase(cif_path)
    # keep ONE atom deterministically
    a = a[[0]]
    # put in a big cubic box
    a.set_cell(np.eye(3)*box, scale_atoms=False); a.center(); a.set_pbc([True,True,True]); a.wrap()
    a.calc = CHGNetCalculator(_chgnet, atom_graph_cutoff=cutoff)
    E = a.get_potential_energy(); mu = E/len(a)
    print(f"{label}: μ_atomic-like = {mu:.6f} eV/atom  (box={box} Å, cutoff={cutoff} Å)")
    return mu


CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu


In [ ]:
# Metals — bulk-like (fixed policy)
mu_Li = mu_from_cif_robust("/home/.../Li_Srinibas.cif", label="Li", solid=True,  cutoff_solid=8.0)
mu_La = mu_from_cif_robust("/home/.../La_Srinibas.cif", label="La", solid=True,  cutoff_solid=8.0)
mu_Zr = mu_from_cif_robust("/home/.../Zr_Srinibas.cif", label="Zr", solid=True,  cutoff_solid=8.0)

# Oxygen — molecular O2
# Best practice (if allowed): ensure O–O ~1.21 Å by isotropic rescale, cutoff=3.0 Å
mu_O  = mu_from_cif_robust("/home/.../O2_Srinibas.cif", label="O", solid=False, cutoff_mol=3.0)

# Lock and reuse everywhere
element_mu = dict(Li=float(mu_Li), La=float(mu_La), Zr=float(mu_Zr), O=float(mu_O))


In [ ]:
"""
Li: μ_model = -1.135527 eV/atom (cutoff=8.0 Å, natoms=16)
CHGNet will run on cpu
La: μ_model = -2.919654 eV/atom (cutoff=8.0 Å, natoms=32)
CHGNet will run on cpu
Zr: μ_model = -3.582404 eV/atom (cutoff=8.0 Å, natoms=16)
CHGNet will run on cpu
O: μ_model = -2.942590 eV/atom (cutoff=3.0 Å, natoms=2)

Using these values of mu, we can calculate the formation energy of LLZO as follows:
"""

import os
from pymatgen.core import Structure
from chgnet.model import CHGNet

# ---- 1. CHGNet model -------------------------------------------------
model = CHGNet.load()   # e_pred is eV/atom !

# ---- 2. elemental chemical potentials (CHGNet column) ----------------
mu = {"Li": mu_Li, "La": mu_La, "Zr": mu_Zr, "O": mu_O}

# ---- 3. read structure ---------------------------------------------------
structure = Structure.from_file("/home/phanim/harshitrawat/summer/LLZO_Srinibas.cif")
natoms = len(structure)
print(f"Structure has {natoms} atoms")
# ---- 4. get total energy -----------------------------------------------
e_pred = model.predict_structure(structure)["e"]  # eV/atom
E_pred = e_pred * natoms                   # eV
print(f"CHGNet total energy = {E_pred:.6f} eV ({e_pred:.6f} eV/atom)")


CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
Structure has 192 atoms
CHGNet total energy = -470.014618 eV (-2.447993 eV/atom)


In [5]:

# ---- 5. compute formation energy ----------------------------------------
E_form = E_pred - sum(mu[str(el)] * structure.composition[el] for el in structure.composition)
print(f"CHGNet formation energy = {E_form:.6f} eV ({E_form/natoms:.6f} eV/atom)")


CHGNet formation energy = 3.453671 eV (0.017988 eV/atom)


In [ ]:
# check what attributes are there in model
print(dir(model))


['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_compute', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_

In [2]:
# === Robust chemical potentials from your CIFs with CHGNet =====================
# Keeps your motif, fixes isolated-atom graphs, and makes μ stable to small CIF edits.

from pathlib import Path
import numpy as np

from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor

from ase import Atoms
from ase.build import make_supercell

from chgnet.model import CHGNet
from chgnet.model.dynamics import CHGNetCalculator

# ----- load CHGNet once -----
_adaptor = AseAtomsAdaptor()
_chgnet = CHGNet.load()


# ---------- low-level helpers ----------
def _load_ase_from_cif_preserve(path: str) -> Atoms:
    """Load CIF via pymatgen -> ASE, preserving motif; wrap fracs into [0,1)."""
    s = Structure.from_file(path)
    s = Structure(lattice=s.lattice, species=s.species,
                  coords=(s.frac_coords % 1.0), coords_are_cartesian=False)
    a = _adaptor.get_atoms(s)
    a.set_pbc([True, True, True])
    a.wrap()
    return a

def _min_periodic_distance(a: Atoms) -> float:
    """Minimum nonzero distance under MIC."""
    D = a.get_all_distances(mic=True)
    np.fill_diagonal(D, np.inf)
    return float(np.nanmin(D))

def _uniform_rescale_to_target_nn(a: Atoms, target=3.0, band=(2.5, 3.2), max_iter=12) -> Atoms:
    """Isotropic rescale so min MIC distance falls in the desired band around 'target'."""
    a = a.copy()
    a.wrap()
    lo, hi = band
    for _ in range(max_iter):
        dmin = _min_periodic_distance(a)
        if np.isfinite(dmin) and lo <= dmin <= hi:
            return a
        if not np.isfinite(dmin) or dmin <= 1e-6:
            scale = 1.2  # expand a bit if pathological
        else:
            # move 70% toward target to avoid oscillation; clamp for stability
            scale = max(0.2, min(5.0, (target / dmin)**0.7))
        a.set_cell(a.cell * scale, scale_atoms=True)
        a.wrap()
    return a  # best-effort

def _deterministic_supercell(a: Atoms, min_len=10.0, min_atoms=16) -> Atoms:
    """Ensure minimum cell lengths and atom count using predictable diagonal replication."""
    a = a.copy()
    # 1) Enforce min cell length by isotropic scale if needed
    Lx, Ly, Lz = a.cell.lengths()
    s = max(min_len / Lx, min_len / Ly, min_len / Lz, 1.0)
    if s > 1.0:
        a.set_cell(a.cell * s, scale_atoms=True)
        a.wrap()

    # 2) Enforce minimum number of atoms via deterministic nx,ny,nz
    n = max(len(a), 1)
    nx = ny = nz = 1
    while nx * ny * nz * n < min_atoms:
        if nx <= ny and nx <= nz:
            nx += 1
        elif ny <= nx and ny <= nz:
            ny += 1
        else:
            nz += 1
    if (nx, ny, nz) != (1, 1, 1):
        a = make_supercell(a, [[nx, 0, 0], [0, ny, 0], [0, 0, nz]])
        a.wrap()
    return a


# ---------- main robust API ----------
def mu_from_cif_robust(
    cif_path: str,
    label: str = "X",
    solid: bool = True,
    cutoff_solid: float = 8.0,
    cutoff_mol: float = 3.0,
    target_nn_solid: float = 3.0,
    nn_band_solid=(2.5, 3.2),
    target_nn_mol: float = 1.5,        # for O2; looser band below
    nn_band_mol=(1.0, 3.5),
    min_len_solid: float = 10.0,
    min_atoms_solid: int = 16,
):
    """
    Compute stable μ (eV/atom) from your CIF using a fixed, deterministic policy.

    solid=True  : bulk-like protocol (Li/La/Zr/solid-O phases)
    solid=False : molecular protocol (O2-in-box). μ returned is *per atom* (½ E(O2)).
    """
    assert Path(cif_path).exists(), f"missing CIF: {cif_path}"
    a = _load_ase_from_cif_preserve(cif_path)

    if solid:
        # bulk-like: target nearest-neighbor ~3 Å,
        # then deterministic supercell + fixed cutoff
        a = _uniform_rescale_to_target_nn(a, target=target_nn_solid, band=nn_band_solid)
        a = _deterministic_supercell(a, min_len=min_len_solid, min_atoms=min_atoms_solid)
        cutoff = cutoff_solid
        per_atom_divisor = len(a)
    else:
        # molecular (e.g., O2 in box): keep diatomic motif, aim for realistic O–O band,
        # no min_atoms inflation; small cutoff so no image bonds
        a = _uniform_rescale_to_target_nn(a, target=target_nn_mol, band=nn_band_mol)
        # keep as-is size; ensure at least ~12 Å box to decouple images
        Lx, Ly, Lz = a.cell.lengths()
        s = max(12.0 / Lx, 12.0 / Ly, 12.0 / Lz, 1.0)
        if s > 1.0:
            a.set_cell(a.cell * s, scale_atoms=True)
            a.wrap()
        cutoff = cutoff_mol
        per_atom_divisor = len(a)  # for O2 this is 2; μ per atom = E/2

    # one or two attempts with small cutoff ladder (deterministic)
    for c in (cutoff, cutoff + 2.0, cutoff + 4.0):
        try:
            a.calc = CHGNetCalculator(_chgnet, atom_graph_cutoff=c)
            E = a.get_potential_energy()  # eV
            mu = E / per_atom_divisor
            print(f"{label}: μ_model = {mu:.6f} eV/atom  "
                  f"(cutoff={c:.1f} Å, natoms={len(a)}, dmin≈{_min_periodic_distance(a):.2f} Å)")
            return mu
        except Exception as e:
            last_err = e

    # Final fallback: gentle 2×2×2 supercell (solids) or slightly bigger box (molecules)
    if solid:
        a2 = make_supercell(a, [[2,0,0],[0,2,0],[0,0,2]])
        a2.wrap()
        c = cutoff + 4.0
        a2.calc = CHGNetCalculator(_chgnet, atom_graph_cutoff=c)
        E = a2.get_potential_energy()
        mu = E / len(a2)
        print(f"{label}: μ_model = {mu:.6f} eV/atom  "
              f"(cutoff={c:.1f} Å, natoms={len(a2)}, dmin≈{_min_periodic_distance(a2):.2f} Å)")
        return mu
    else:
        # molecular: expand box a bit and retry once
        a2 = a.copy()
        a2.set_cell(a2.cell * 1.25, scale_atoms=False); a2.center(); a2.wrap()
        c = cutoff + 2.0
        a2.calc = CHGNetCalculator(_chgnet, atom_graph_cutoff=c)
        E = a2.get_potential_energy()
        mu = E / len(a2)
        print(f"{label}: μ_model = {mu:.6f} eV/atom  "
              f"(cutoff={c:.1f} Å, natoms={len(a2)}, dmin≈{_min_periodic_distance(a2):.2f} Å)")
        return mu


# ---------- convenience wrappers (choose one policy and keep it fixed) ----------
def mu_bulk_like(cif_path: str, label="X",
                 cutoff=8.0, target_nn=3.0, band=(2.5, 3.2),
                 min_len=10.0, min_atoms=16):
    return mu_from_cif_robust(
        cif_path, label=label, solid=True,
        cutoff_solid=cutoff, target_nn_solid=target_nn, nn_band_solid=band,
        min_len_solid=min_len, min_atoms_solid=min_atoms
    )

def mu_molecular_O2(cif_path: str, label="O",
                    cutoff=3.0, target_nn=1.5, band=(1.0, 3.5)):
    # returns μ per atom = ½ E(O2)
    return mu_from_cif_robust(
        cif_path, label=label, solid=False,
        cutoff_mol=cutoff, target_nn_mol=target_nn, nn_band_mol=band
    )


# ---------- example calls (edit your absolute paths) ----------
# mu_Li = mu_bulk_like("/home/phanim/harshitrawat/summer/Li_Srinibas.cif", label="Li")
# mu_La = mu_bulk_like("/home/phanim/harshitrawat/summer/La_Srinibas.cif", label="La")
# mu_Zr = mu_bulk_like("/home/phanim/harshitrawat/summer/Zr_Srinibas.cif", label="Zr")
# mu_O  = mu_molecular_O2("/home/phanim/harshitrawat/summer/O2_Srinibas.cif", label="O")

# # lock them for reuse:
# import json
# element_mu = dict(Li=float(mu_Li), La=float(mu_La), Zr=float(mu_Zr), O=float(mu_O))
# with open("element_mu_chgnet_locked.json", "w") as f:
#     json.dump(element_mu, f, indent=2)
# print(element_mu)


CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/chgnet/model/model.py:673: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load

In [3]:
# ---------- example calls (edit your absolute paths) ----------
mu_Li = mu_bulk_like("/home/phanim/harshitrawat/summer/Li_Srinibas.cif", label="Li")
mu_La = mu_bulk_like("/home/phanim/harshitrawat/summer/La_Srinibas.cif", label="La")
mu_Zr = mu_bulk_like("/home/phanim/harshitrawat/summer/Zr_Srinibas.cif", label="Zr")
mu_O  = mu_molecular_O2("/home/phanim/harshitrawat/summer/O2_Srinibas.cif", label="O")


CHGNet will run on cpu
Li: μ_model = -1.135527 eV/atom  (cutoff=8.0 Å, natoms=16, dmin≈4.33 Å)
CHGNet will run on cpu


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 16 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


La: μ_model = -2.919654 eV/atom  (cutoff=8.0 Å, natoms=32, dmin≈4.95 Å)
CHGNet will run on cpu


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


Zr: μ_model = -3.582404 eV/atom  (cutoff=8.0 Å, natoms=16, dmin≈4.93 Å)
CHGNet will run on cpu
O: μ_model = -2.942590 eV/atom  (cutoff=3.0 Å, natoms=2, dmin≈2.30 Å)
